In [1]:
%load_ext watermark
%watermark -i -h -m -v -p numpy,MotionClouds 

Python implementation: CPython
Python version       : 3.12.6
IPython version      : 8.27.0

numpy       : 1.26.4
MotionClouds: 20220927

Compiler    : Clang 16.0.0 (clang-1600.0.26.3)
OS          : Darwin
Release     : 24.0.0
Machine     : arm64
Processor   : arm
CPU cores   : 10
Architecture: 64bit

Hostname: obiwan.local



# pilot: one B_sf some B_theta many thetas

In [2]:
%rm -fr img_pilot_folder

In [4]:
import os
data_folder = 'img_pilot_folder'
os.makedirs(data_folder, exist_ok=True)

print('\n Initializing')

# parameters 
import MotionClouds as mc
import numpy as np

N_B_theta = 3
N_B_sf = 1
N_repet = 1
N_thetas = 12

B_thetas = np.pi/3 * np.linspace(0, 1, N_B_theta+1)[1:]
print('B_thetas = ', B_thetas)
B_sfs = [mc.B_sf] #* np.logspace(-1, -1, N_B_sf, base=2)
print('B_sfs = ', B_sfs)
theta_max = np.pi/8
thetas = np.linspace(-theta_max, theta_max, N_thetas)

# generate all clouds
import imageio
def generate_random_cloud(i_trial, theta, B_theta, B_sf, seed, downscale = 1):
    # fname = f'{data_folder}/theta_{theta}_B_theta_{B_theta}_B_sf_{B_sf}_seed_{seed}.png'
    fname = f'{data_folder}/{i_trial}.png'
    if not os.path.isfile(fname):
        fx, fy, ft = mc.get_grids(mc.N_X/downscale, mc.N_Y/downscale, 1)
        mc_i = mc.envelope_gabor(fx, fy, ft, V_X=0., V_Y=0., B_sf=B_sf,
                                 B_V=0, theta=np.pi/2-theta, B_theta=B_theta)
        im = mc.random_cloud(mc_i, seed=seed)
        im = (mc.rectif(im) * 255).astype('uint8')
        imageio.imwrite(fname, im[:, :, 0])
    return fname


all_conditions = [(i_repet, i_theta, i_B_theta, i_B_sf) 
                  for i_repet in range(N_repet) 
                  for i_theta in range(N_thetas) 
                  for i_B_theta in range(N_B_theta) 
                  for i_B_sf in range(N_B_sf)]
N_total_trials = len(all_conditions)
ind = np.random.permutation(N_total_trials)

seed = 2024
np.random.seed(seed)
parameters = []
for i_trial in range(N_total_trials):
    i_repet, i_theta, i_B_theta, i_B_sf = all_conditions[ind[i_trial]]

    fname = generate_random_cloud(i_trial, thetas[i_theta], 
                               B_theta=B_thetas[i_B_theta], 
                               B_sf=B_sfs[i_B_sf], 
                               seed=seed+i_trial)
    parameters.append({'fname':fname, 'theta': thetas[i_theta], 'B_theta': B_thetas[i_B_theta], 'B_sf': B_sfs[i_B_sf], 'seed': seed+i_trial, 'i_trial': i_trial})
    print(f"          {{stimulus: '{fname}'}},",

)


import json
with open('img_pilot_folder/parameters.json', 'w') as f:
    json.dump(parameters, f)



 Initializing
B_thetas =  [0.34906585 0.6981317  1.04719755]
B_sfs =  [0.1]
          {stimulus: 'img_pilot_folder/0.png'},
          {stimulus: 'img_pilot_folder/1.png'},
          {stimulus: 'img_pilot_folder/2.png'},
          {stimulus: 'img_pilot_folder/3.png'},
          {stimulus: 'img_pilot_folder/4.png'},
          {stimulus: 'img_pilot_folder/5.png'},
          {stimulus: 'img_pilot_folder/6.png'},
          {stimulus: 'img_pilot_folder/7.png'},
          {stimulus: 'img_pilot_folder/8.png'},
          {stimulus: 'img_pilot_folder/9.png'},
          {stimulus: 'img_pilot_folder/10.png'},
          {stimulus: 'img_pilot_folder/11.png'},
          {stimulus: 'img_pilot_folder/12.png'},
          {stimulus: 'img_pilot_folder/13.png'},
          {stimulus: 'img_pilot_folder/14.png'},
          {stimulus: 'img_pilot_folder/15.png'},
          {stimulus: 'img_pilot_folder/16.png'},
          {stimulus: 'img_pilot_folder/17.png'},
          {stimulus: 'img_pilot_folder/18.png'},
  

In [17]:
all_conditions = [(i_repet, i_theta, i_B_theta, i_B_sf) for i_repet in range(N_repet) for i_theta in range(N_thetas) for i_B_theta in range(N_B_theta) for i_B_sf in range(N_B_sf)]
N_total_trials = len(all_conditions)
ind = np.random.permutation(N_total_trials)

In [ ]:
N_thetas, N_B_theta, N_total_trials

In [ ]:
all_conditions